In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [2]:
flow_with_sqrt=pd.read_csv('../data/flow_df_with_sqrt.csv')
flow_no_sqrt=pd.read_csv('../data/flow_df_not_sqrt.csv')
print(flow_with_sqrt.shape,flow_no_sqrt.shape)
print(flow_with_sqrt.columns,flow_no_sqrt.columns)
tmp=flow_no_sqrt[['date_dt','address_str','flow_out_not_sqrt','flow_in_not_sqrt']]
flow=flow_with_sqrt.merge(tmp,on=['date_dt','address_str'],how='left')

(28322, 15) (28322, 15)
Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'start_address', 'flow_out_sqrt', 'end_address', 'flow_in_sqrt'],
      dtype='object') Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'start_address', 'flow_out_not_sqrt', 'end_address',
       'flow_in_not_sqrt'],
      dtype='object')


In [3]:
del flow['start_address']
del flow['end_address']

In [4]:
print(flow.columns)

Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt',
       'flow_in_not_sqrt'],
      dtype='object')


In [5]:
f_col=['flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt','flow_in_not_sqrt']
for fea in f_col:
    flow[fea]=flow[fea].apply(lambda x:round(x,6))

In [6]:
flow['in_bigger_out']=np.where(flow['flow_in']>flow['flow_out'],1,0)

In [7]:
flow['in_bigger_out_nosqrt']=np.where(flow['flow_in_not_sqrt']>flow['flow_out_not_sqrt'],1,0)

In [8]:
flow['in_bigger_out_sqrt']=np.where(flow['flow_in_sqrt']>flow['flow_out_sqrt'],1,0)

In [9]:
flow['all_big']=0
flow.loc[(flow['in_bigger_out']==flow['in_bigger_out_nosqrt'])&(flow['in_bigger_out']==flow['in_bigger_out_sqrt']),'all_big']=1

In [10]:
gc.collect()

38

In [11]:
pair_col=[('flow_out_sqrt', 'flow_in_sqrt'), 
          ('flow_out_not_sqrt','flow_in_not_sqrt')]
for i in range(2):
    lst=pair_col[i]
    fea_name=str(i)+'_mean'
    flow[fea_name]=round((flow[lst[0]]+flow[lst[1]])/2,6)
    
for i in range(2):
    lst=pair_col[i]
    fea_name=str(i)+'_sub'
    flow[fea_name]=round((flow[lst[0]]/flow[lst[1]]),6)
    
for i in range(2):
    lst=pair_col[i]
    fea_name=str(i)+'_del'
    flow[fea_name]=round((flow[lst[0]]-flow[lst[1]]),6)

In [12]:
print(pd.isnull(flow).sum())

address                    0
address_str                0
city_code                  0
date_dt                    0
day                        0
district_code              0
dwell                   1470
flow_in                 1470
flow_out                1470
month                      0
weekd                      0
flow_out_sqrt              0
flow_in_sqrt               0
flow_out_not_sqrt          0
flow_in_not_sqrt           0
in_bigger_out              0
in_bigger_out_nosqrt       0
in_bigger_out_sqrt         0
all_big                    0
0_mean                     0
1_mean                     0
0_sub                      0
1_sub                      0
0_del                      0
1_del                      0
dtype: int64


In [13]:
print(flow.shape) #(28322, ?)

print(len(flow[flow['dwell'].isnull()])) #1470

(28322, 25)
1470


In [14]:
#把已有的比例赋值给没有的
true_col=['dwell','flow_in','flow_out']
for i in range(3):
    for j in range(i+1,3):
        name1=true_col[i]+'_'+true_col[j]
        flow[name1]=round(flow[true_col[i]]/flow[true_col[j]],6)
        
        name2=true_col[i]+'_mean_'+true_col[j]
        flow[name2]=round((flow[true_col[i]]+flow[true_col[j]])/2,6)

print(flow.shape) #(28322, ?)
print(len(flow[flow['dwell'].isnull()])) #1470

(28322, 31)
1470


In [15]:
to_col=['dwell_flow_in', 'dwell_mean_flow_in','dwell_flow_out', 'dwell_mean_flow_out', 'flow_in_flow_out',
       'flow_in_mean_flow_out']

for fea in to_col:
    train=flow[flow['dwell'].notnull()][['address','day','weekd',fea]]
    tmp=train.groupby(['address','day','weekd'])[fea].mean().reset_index(name=fea+'_n')
    flow=flow.merge(tmp,on=['address','day','weekd'],how='left')
flow=flow.drop(to_col,axis=1)
del tmp
print(flow.shape) #(28322, ?)
print(len(flow[flow['dwell'].isnull()])) #1470
gc.collect()

(28322, 31)
1470


131

In [16]:
# #这个评估标准是大赛规定的
def rmsle(y_true, y_pred):
    return 'RMSLE', round(np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))),6), False

In [17]:
train=flow[flow['dwell'].notnull()]
print(train.shape,train.columns)
test=flow[flow['dwell'].isnull()]
print(test.shape,test.columns)

(26852, 31) Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt',
       'flow_in_not_sqrt', 'in_bigger_out', 'in_bigger_out_nosqrt',
       'in_bigger_out_sqrt', 'all_big', '0_mean', '1_mean', '0_sub', '1_sub',
       '0_del', '1_del', 'dwell_flow_in_n', 'dwell_mean_flow_in_n',
       'dwell_flow_out_n', 'dwell_mean_flow_out_n', 'flow_in_flow_out_n',
       'flow_in_mean_flow_out_n'],
      dtype='object')
(1470, 31) Index(['address', 'address_str', 'city_code', 'date_dt', 'day',
       'district_code', 'dwell', 'flow_in', 'flow_out', 'month', 'weekd',
       'flow_out_sqrt', 'flow_in_sqrt', 'flow_out_not_sqrt',
       'flow_in_not_sqrt', 'in_bigger_out', 'in_bigger_out_nosqrt',
       'in_bigger_out_sqrt', 'all_big', '0_mean', '1_mean', '0_sub', '1_sub',
       '0_del', '1_del', 'dwell_flow_in_n', 'dwell_mean_flow_in_n',
       'dwell_flow_o

In [18]:
feature=['address','day','weekd', 'flow_out_sqrt', 'flow_in_sqrt','flow_out_not_sqrt', 
         'flow_in_not_sqrt', 'in_bigger_out',
       'in_bigger_out_nosqrt', 'in_bigger_out_sqrt', 'all_big', '0_mean',
       '1_mean', '0_sub', '1_sub', '0_del', '1_del', 'dwell_flow_in_n',
       'dwell_mean_flow_in_n', 'dwell_flow_out_n', 'dwell_mean_flow_out_n',
       'flow_in_flow_out_n', 'flow_in_mean_flow_out_n']

label=['flow_in','flow_out','dwell']

In [19]:
traindf=train[train['date_dt']<20180210]
valdf=train[train['date_dt']>=20180210]
print(traindf.shape,valdf.shape)

(24892, 31) (1960, 31)


In [20]:
result = pd.DataFrame(columns=['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out'])

In [21]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [22]:
city=list(set(flow['city_code']))

In [27]:
#有7个city
for ct in city:
    for y in label:
        #已知训练集
        train_x= traindf[traindf['city_code']==ct][feature]
        train_y = traindf[traindf['city_code']==ct][y]
        ##已知验证集
        val_x = valdf[valdf['city_code']==ct][feature]
        val_y = valdf[valdf['city_code']==ct][y]
        
        #由于每一个ad对应的只有274个数据，所以不用再分了，直接验证rmsle并预测
        gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                            objective='regression',
                            num_leaves=100,
                            subsample=0.8, colsample_bytree=0.8, 
                            learning_rate=0.05,
                            n_estimators=2000)

        gbm.fit(train_x, train_y, 
                eval_set=[(train_x,train_y),(val_x, val_y)],
                eval_metric='rmse',
                early_stopping_rounds=100)

        y_pred = gbm.predict(val_x, num_iteration=gbm.best_iteration_)
        print('The rmsle on val is:', rmsle(val_y, y_pred)[1])
        print("====================================================================")

        gc.collect()
        
        test[y] = gbm.predict(test[feature])
        
    result = pd.concat([result,test[result.columns]])

[1]	training's rmse: 93.8026	valid_1's rmse: 95.545
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 89.2817	valid_1's rmse: 92.0891
[3]	training's rmse: 84.9874	valid_1's rmse: 88.9563
[4]	training's rmse: 80.9378	valid_1's rmse: 86.3909
[5]	training's rmse: 77.0773	valid_1's rmse: 83.6333
[6]	training's rmse: 73.4195	valid_1's rmse: 80.933
[7]	training's rmse: 69.9519	valid_1's rmse: 78.3771
[8]	training's rmse: 66.6692	valid_1's rmse: 76.0349
[9]	training's rmse: 63.5617	valid_1's rmse: 73.8813
[10]	training's rmse: 60.6163	valid_1's rmse: 71.8711
[11]	training's rmse: 57.824	valid_1's rmse: 69.9902
[12]	training's rmse: 55.1849	valid_1's rmse: 68.2513
[13]	training's rmse: 52.6868	valid_1's rmse: 66.6698
[14]	training's rmse: 50.3314	valid_1's rmse: 65.5831
[15]	training's rmse: 48.0985	valid_1's rmse: 64.196
[16]	training's rmse: 45.9863	valid_1's rmse: 62.9145
[17]	training's rmse: 43.9932	valid_1's rmse: 61.9418
[18]	training's rmse: 42.1147	va

[1]	training's rmse: 96.7535	valid_1's rmse: 100.964
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 92.0849	valid_1's rmse: 97.3504
[3]	training's rmse: 87.6469	valid_1's rmse: 94.0718
[4]	training's rmse: 83.4605	valid_1's rmse: 91.1142
[5]	training's rmse: 79.4744	valid_1's rmse: 88.389
[6]	training's rmse: 75.6907	valid_1's rmse: 85.5666
[7]	training's rmse: 72.1084	valid_1's rmse: 82.9926
[8]	training's rmse: 68.7171	valid_1's rmse: 80.571
[9]	training's rmse: 65.4988	valid_1's rmse: 78.4211
[10]	training's rmse: 62.4569	valid_1's rmse: 76.3665
[11]	training's rmse: 59.5645	valid_1's rmse: 74.4952
[12]	training's rmse: 56.839	valid_1's rmse: 72.6381
[13]	training's rmse: 54.2467	valid_1's rmse: 71.0872
[14]	training's rmse: 51.8108	valid_1's rmse: 69.9633
[15]	training's rmse: 49.4892	valid_1's rmse: 68.6794
[16]	training's rmse: 47.3006	valid_1's rmse: 67.3187
[17]	training's rmse: 45.226	valid_1's rmse: 66.1931
[18]	training's rmse: 43.2717	va

[1]	training's rmse: 79.3854	valid_1's rmse: 89.2386
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 75.5041	valid_1's rmse: 86.5292
[3]	training's rmse: 71.82	valid_1's rmse: 84.0434
[4]	training's rmse: 68.3252	valid_1's rmse: 81.7811
[5]	training's rmse: 65.0084	valid_1's rmse: 79.6723
[6]	training's rmse: 61.8655	valid_1's rmse: 77.5077
[7]	training's rmse: 58.8707	valid_1's rmse: 75.7627
[8]	training's rmse: 56.0415	valid_1's rmse: 73.8489
[9]	training's rmse: 53.3472	valid_1's rmse: 72.2969
[10]	training's rmse: 50.7945	valid_1's rmse: 70.8133
[11]	training's rmse: 48.3736	valid_1's rmse: 69.5368
[12]	training's rmse: 46.0788	valid_1's rmse: 68.0401
[13]	training's rmse: 43.8956	valid_1's rmse: 66.9877
[14]	training's rmse: 41.8258	valid_1's rmse: 65.9982
[15]	training's rmse: 39.8623	valid_1's rmse: 65.1164
[16]	training's rmse: 38.0127	valid_1's rmse: 64.084
[17]	training's rmse: 36.2511	valid_1's rmse: 63.2419
[18]	training's rmse: 34.5816	v

The rmsle on val is: 0.348087
[1]	training's rmse: 1092.93	valid_1's rmse: 1310.57
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 1039.09	valid_1's rmse: 1257.74
[3]	training's rmse: 987.946	valid_1's rmse: 1207.69
[4]	training's rmse: 939.438	valid_1's rmse: 1159.96
[5]	training's rmse: 893.387	valid_1's rmse: 1114.39
[6]	training's rmse: 849.63	valid_1's rmse: 1071.22
[7]	training's rmse: 808.114	valid_1's rmse: 1030.66
[8]	training's rmse: 768.71	valid_1's rmse: 992.399
[9]	training's rmse: 731.352	valid_1's rmse: 956.215
[10]	training's rmse: 695.89	valid_1's rmse: 921.818
[11]	training's rmse: 662.236	valid_1's rmse: 889.477
[12]	training's rmse: 630.328	valid_1's rmse: 858.683
[13]	training's rmse: 600.084	valid_1's rmse: 829.973
[14]	training's rmse: 571.378	valid_1's rmse: 802.76
[15]	training's rmse: 544.198	valid_1's rmse: 777.147
[16]	training's rmse: 518.395	valid_1's rmse: 752.489
[17]	training's rmse: 493.968	valid_1's rmse: 729.354
[1

[169]	training's rmse: 85.3174	valid_1's rmse: 394.578
[170]	training's rmse: 85.0857	valid_1's rmse: 394.566
[171]	training's rmse: 84.8613	valid_1's rmse: 394.562
[172]	training's rmse: 84.6917	valid_1's rmse: 394.676
[173]	training's rmse: 84.5537	valid_1's rmse: 394.815
[174]	training's rmse: 84.3592	valid_1's rmse: 394.861
[175]	training's rmse: 84.1922	valid_1's rmse: 394.995
[176]	training's rmse: 84.0061	valid_1's rmse: 395.19
[177]	training's rmse: 83.8198	valid_1's rmse: 395.316
[178]	training's rmse: 83.6783	valid_1's rmse: 395.477
[179]	training's rmse: 83.5366	valid_1's rmse: 395.788
[180]	training's rmse: 83.3803	valid_1's rmse: 396.176
[181]	training's rmse: 83.2046	valid_1's rmse: 396.18
[182]	training's rmse: 83.0198	valid_1's rmse: 396.231
[183]	training's rmse: 82.865	valid_1's rmse: 396.373
[184]	training's rmse: 82.711	valid_1's rmse: 396.655
[185]	training's rmse: 82.5288	valid_1's rmse: 396.684
[186]	training's rmse: 82.362	valid_1's rmse: 396.866
Early stopping,

...

[122]	training's rmse: 9.91015	valid_1's rmse: 19.967
[123]	training's rmse: 9.88832	valid_1's rmse: 19.9787
[124]	training's rmse: 9.8688	valid_1's rmse: 19.9903
[125]	training's rmse: 9.84651	valid_1's rmse: 20.0248
[126]	training's rmse: 9.82896	valid_1's rmse: 20.0321
[127]	training's rmse: 9.81016	valid_1's rmse: 20.0528
[128]	training's rmse: 9.79273	valid_1's rmse: 20.0638
[129]	training's rmse: 9.77554	valid_1's rmse: 20.0791
[130]	training's rmse: 9.7517	valid_1's rmse: 20.13
[131]	training's rmse: 9.73439	valid_1's rmse: 20.1632
[132]	training's rmse: 9.71666	valid_1's rmse: 20.1757
[133]	training's rmse: 9.70048	valid_1's rmse: 20.2008
[134]	training's rmse: 9.67962	valid_1's rmse: 20.2599
[135]	training's rmse: 9.66302	valid_1's rmse: 20.2722
[136]	training's rmse: 9.64327	valid_1's rmse: 20.3015
[137]	training's rmse: 9.62537	valid_1's rmse: 20.3557
[138]	training's rmse: 9.60952	valid_1's rmse: 20.3642
[139]	training's rmse: 9.59354	valid_1's rmse: 20.3676
[140]	training'

[119]	training's rmse: 5.82323	valid_1's rmse: 18.5367
[120]	training's rmse: 5.81158	valid_1's rmse: 18.5536
[121]	training's rmse: 5.79769	valid_1's rmse: 18.5773
[122]	training's rmse: 5.78496	valid_1's rmse: 18.5935
[123]	training's rmse: 5.77089	valid_1's rmse: 18.5923
[124]	training's rmse: 5.75765	valid_1's rmse: 18.6094
[125]	training's rmse: 5.74211	valid_1's rmse: 18.6236
[126]	training's rmse: 5.73186	valid_1's rmse: 18.6289
[127]	training's rmse: 5.71785	valid_1's rmse: 18.6305
[128]	training's rmse: 5.70725	valid_1's rmse: 18.6405
[129]	training's rmse: 5.69756	valid_1's rmse: 18.644
[130]	training's rmse: 5.68226	valid_1's rmse: 18.6591
[131]	training's rmse: 5.67141	valid_1's rmse: 18.6739
[132]	training's rmse: 5.66026	valid_1's rmse: 18.6753
[133]	training's rmse: 5.64412	valid_1's rmse: 18.6814
[134]	training's rmse: 5.63374	valid_1's rmse: 18.6835
[135]	training's rmse: 5.6187	valid_1's rmse: 18.6865
[136]	training's rmse: 5.60876	valid_1's rmse: 18.692
[137]	trainin

The rmsle on val is: 0.24002
[1]	training's rmse: 91.7942	valid_1's rmse: 84.802
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 87.2614	valid_1's rmse: 80.7537
[3]	training's rmse: 82.9603	valid_1's rmse: 77.2077
[4]	training's rmse: 78.871	valid_1's rmse: 73.7367
[5]	training's rmse: 74.9869	valid_1's rmse: 70.5087
[6]	training's rmse: 71.2999	valid_1's rmse: 67.2752
[7]	training's rmse: 67.799	valid_1's rmse: 64.2012
[8]	training's rmse: 64.4782	valid_1's rmse: 61.4581
[9]	training's rmse: 61.3235	valid_1's rmse: 58.7307
[10]	training's rmse: 58.3285	valid_1's rmse: 56.1934
[11]	training's rmse: 55.4914	valid_1's rmse: 53.9511
[12]	training's rmse: 52.7949	valid_1's rmse: 51.8352
[13]	training's rmse: 50.2358	valid_1's rmse: 49.7111
[14]	training's rmse: 47.8058	valid_1's rmse: 47.764
[15]	training's rmse: 45.5019	valid_1's rmse: 46.078
[16]	training's rmse: 43.3136	valid_1's rmse: 44.4209
[17]	training's rmse: 41.2397	valid_1's rmse: 42.7976
[18]

The rmsle on val is: 0.250155
[1]	training's rmse: 46.0557	valid_1's rmse: 25.6517
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 44.3646	valid_1's rmse: 26.2281
[3]	training's rmse: 42.7565	valid_1's rmse: 26.7736
[4]	training's rmse: 41.2654	valid_1's rmse: 27.3316
[5]	training's rmse: 39.841	valid_1's rmse: 27.8894
[6]	training's rmse: 38.5346	valid_1's rmse: 28.3886
[7]	training's rmse: 37.2829	valid_1's rmse: 28.9425
[8]	training's rmse: 36.0952	valid_1's rmse: 29.6581
[9]	training's rmse: 34.9923	valid_1's rmse: 30.2204
[10]	training's rmse: 33.9616	valid_1's rmse: 30.6601
[11]	training's rmse: 32.9793	valid_1's rmse: 31.3213
[12]	training's rmse: 32.0727	valid_1's rmse: 31.9336
[13]	training's rmse: 31.2244	valid_1's rmse: 32.5095
[14]	training's rmse: 30.4338	valid_1's rmse: 33.095
[15]	training's rmse: 29.7017	valid_1's rmse: 33.5752
[16]	training's rmse: 29.0353	valid_1's rmse: 34.0362
[17]	training's rmse: 28.4038	valid_1's rmse: 34.4816


[1]	training's rmse: 42.2016	valid_1's rmse: 27.0634
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 40.6742	valid_1's rmse: 27.5932
[3]	training's rmse: 39.2493	valid_1's rmse: 27.9928
[4]	training's rmse: 37.9209	valid_1's rmse: 28.4951
[5]	training's rmse: 36.674	valid_1's rmse: 28.8948
[6]	training's rmse: 35.4974	valid_1's rmse: 29.3744
[7]	training's rmse: 34.403	valid_1's rmse: 29.7577
[8]	training's rmse: 33.389	valid_1's rmse: 30.1844
[9]	training's rmse: 32.426	valid_1's rmse: 30.6417
[10]	training's rmse: 31.5164	valid_1's rmse: 31.2421
[11]	training's rmse: 30.6875	valid_1's rmse: 31.6668
[12]	training's rmse: 29.9116	valid_1's rmse: 32.0154
[13]	training's rmse: 29.1886	valid_1's rmse: 32.4709
[14]	training's rmse: 28.5218	valid_1's rmse: 32.8634
[15]	training's rmse: 27.9077	valid_1's rmse: 33.1235
[16]	training's rmse: 27.3316	valid_1's rmse: 33.5373
[17]	training's rmse: 26.7857	valid_1's rmse: 33.9358
[18]	training's rmse: 26.2956	va

# 由上可见，按照city分，效果非常差

In [24]:
# result['date_dt'] = result['date_dt'].astype(int)
# result = result[['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in', 'flow_out']]
# print(pd.isnull(result).sum())
# print(result.shape)
# gc.collect()

date_dt          0
city_code        0
district_code    0
dwell            0
flow_in          0
flow_out         0
dtype: int64
(10290, 6)


24

In [25]:
# result.to_csv('..\data\predictionV5_1.csv', index=False, header=None)